> Fundação Getúlio Vargas - RJ <br>
> Escola de Matemática Aplicada (EMAp) <br>
> Graduação em Ciência de Dados e Inteligência Artificial <br>
> Alunos: Gianlucca Devigili e Maisa de O. Fraiz <br>
# Projetos em Ciência de Dados - A1

## Introdução

## Instruções de Execução

## Carga e Tratamento de Dados

## Modelos

### Modelagem até a A1

### Multi Output Chaining
[note] adicionar aqui o teste falho de multi output chaining, explicar porque deu errado e mostrar que os resultados de um target não dependem dos demais. Além disso adicionar a correlação.

### Multi Output Regressor e Modelos Finais